In [2]:
import os
import json
import pandas as pd
import traceback

In [3]:
from langchain.chat_models import ChatOpenAI

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
KEY = os.getenv("OPENAI_API_KEY")

In [10]:
llm = ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.5)


In [ ]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback


In [43]:
RESPONSE_JSON = {
    "1":{
        "mcq":"multiple choice questions",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    "2":{
        "mcq":"multiple choice questions",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    "3":{
        "mcq":"multiple choice questions",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    }
}

In [44]:
TEMPLATE = """
Text:{text}
You are an expert MCQ maker, Given the above text, it is your job to \
create a qui of {number} multiple choice questions for {subject} students in {tone}.
Make sure the questsion are not repeated and check all the questions to be conforming the text as well.
Make sure to format you response like RESPONSE_JSON and use it as a guide.\
Ensure to make {number} MCQs
###Response JSON
{response_json}
"""

In [45]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [46]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [47]:
TEMPLATE2 = """
You are an expert English grammar checker and writer. Give a multiple choice quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complate analysis of the quiz. Only use at max 50 words for complexity check. \
If the quiz questions which needs to be changed and change the tone such that it is proper for student abilities. \
Quiz_MCQs:
{quiz}

Check from an expert English writer of the above quiz:
"""    

In [48]:
quiz_evaluate_prompt = PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE2
)

In [49]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluate_prompt, output_key="review", verbose=True)

In [50]:
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=["text", "number", "subject", "tone", "response_json"],
    output_variables=["quiz", "review"],
    verbose=True
)

In [51]:
file_path = "/Users/prayagsonar/Downloads/Projects/MCQGEN/data.txt"

In [52]:
with open(file_path, "r") as file:
    TEXT = file.read()

In [ ]:
json.dumps(RESPONSE_JSON)

In [59]:
NUMBER=5
SUBJECT="Generative AI"
TONE="Simple"

In [ ]:
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
        }
    )

In [ ]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost: ${cb.total_cost:.4f}")

In [ ]:
response

In [93]:
quiz= response.get("quiz")

In [94]:
quiz = quiz

In [95]:
import json

quiz_table_data=[]
quiz_data = json.loads(quiz)
for key,value in quiz_data.items():
    mcq = value.get("mcq")
    options=" | ".join(
        [f"{option}:{option_value}" for option, option_value in value.get("options").items()]
    )
    correct = value["correct"]
    quiz_table_data.append({
        "MCQ": mcq,
        "Options": options,
        "Correct": correct
    })

In [96]:
quiz = pd.DataFrame(quiz_table_data)

In [97]:
quiz.to_csv("GenerativeAI.csv", index=False)